In [ ]:
# ============================================
# Colab — Diversidade de séries (CRID) × geografia/latitude
# ============================================

!pip install geopandas matplotlib pandas numpy shapely fiona statsmodels --quiet

import pandas as pd, numpy as np, geopandas as gpd
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
from scipy import stats
import statsmodels.api as sm
import zipfile, os, glob, io, warnings
from google.colab import files

warnings.filterwarnings("ignore")

DPI_OUT   = 600
ROBIN_CRS = "ESRI:54030"   # para mapas
WGS84     = "EPSG:4326"    # para latitudes
MISSING_COLOR = "#D3D3D3"

# -----------------------------
# 1) Upload do dataset principal
# -----------------------------
print("📁 Selecione seu arquivo principal (ex.: 'quarto_file_input_world_map.csv')")
up = files.upload()
data_path = list(up.keys())[0]
df_raw = pd.read_csv(data_path)

# Checagens mínimas (ajuste os nomes aqui se preciso)
COUNTRY_COL = "country_corrigido"
CRID_COL    = "CRID"

assert COUNTRY_COL in df_raw.columns, f"Coluna '{COUNTRY_COL}' não encontrada."
assert CRID_COL in df_raw.columns, f"Coluna '{CRID_COL}' não encontrada."

df = df_raw[[COUNTRY_COL, CRID_COL]].copy()
df[COUNTRY_COL] = df[COUNTRY_COL].astype(str).str.strip()
df[CRID_COL]    = df[CRID_COL].astype(str).str.strip()

# Harmonização básica de nomes de países
rename_map = {
    "Lybia":"Libya","Cote d'Ivoire":"Côte d’Ivoire","Ivory Coast":"Côte d’Ivoire",
    "USA":"United States of America","United States":"United States of America",
    "Russia":"Russian Federation","South Korea":"Republic of Korea","North Korea":"Dem. Rep. Korea",
    "Czech Republic":"Czechia","UAE":"United Arab Emirates"
}
df[COUNTRY_COL] = df[COUNTRY_COL].replace(rename_map)

# --------------------------------------
# 2) Cálculo da diversidade (CRIDs únicos)
# --------------------------------------
diversity = (df.dropna(subset=[COUNTRY_COL, CRID_COL])
               .drop_duplicates([COUNTRY_COL, CRID_COL])
               .groupby(COUNTRY_COL)
               .size()
               .reset_index(name="n_CRIDs"))

# --------------------------------------
# 3) Upload do shapefile de países (NE 110m)
# --------------------------------------
print("\n🌍 Selecione 'ne_110m_admin_0_countries.zip' (Natural Earth)")
up_world = files.upload()
zip_world = [k for k in up_world if k.lower().endswith(".zip")]
assert zip_world, "Envie o .zip de países."

os.makedirs("/content/ne_countries", exist_ok=True)
with zipfile.ZipFile(zip_world[0], "r") as z:
    z.extractall("/content/ne_countries")

shp_candidates = glob.glob("/content/ne_countries/**/*.shp", recursive=True)
shp_countries = [p for p in shp_candidates if "admin_0_countries" in os.path.basename(p).lower()] or shp_candidates
assert shp_countries, "Não encontrei .shp de países no zip."

world = gpd.read_file(shp_countries[0])

# Nome do campo com o nome do país no shapefile
NAME_WORLD = next((c for c in ["name","NAME","NAME_EN","ADMIN"] if c in world.columns), None)
assert NAME_WORLD is not None, f"Coluna de nome não encontrada no shapefile: {world.columns.tolist()}"

# Mantém a Antártida (NÃO removemos)
# Coordenadas para latitude: garanta WGS84
if str(world.crs).lower() != "epsg:4326":
    world_wgs = world.to_crs(WGS84)
else:
    world_wgs = world.copy()

# Ponto representativo (garante ponto dentro do polígono)
world_wgs["rep_pt"] = world_wgs.representative_point()
world_wgs["lat"] = world_wgs["rep_pt"].y
world_wgs["lon"] = world_wgs["rep_pt"].x
world_wgs["abs_lat"] = world_wgs["lat"].abs()

# --------------------------------------
# 4) Merge diversidade + geometrias
# --------------------------------------
world_m = world_wgs.merge(diversity, left_on=NAME_WORLD, right_on=COUNTRY_COL, how="left")
world_m["n_CRIDs"] = world_m["n_CRIDs"].fillna(0).astype(int)

# Zonas climáticas aproximadas por latitude (sem depender de dados externos)
def lat_zone(lat):
    a = abs(lat)
    if a < 23.5:   return "Tropical"
    if a < 35.0:   return "Subtropical"
    if a < 55.0:   return "Temperate"
    return "Boreal/Polar"

world_m["LatZone"] = world_m["lat"].apply(lat_zone)

# --------------------------------------
# 5) Estatística: correlação e GLM
# --------------------------------------
df_stats = world_m[world_m["n_CRIDs"] > 0][["abs_lat","n_CRIDs"]].copy()
spearman_r, spearman_p = (np.nan, np.nan)
if len(df_stats) >= 3:
    spearman_r, spearman_p = stats.spearmanr(df_stats["abs_lat"], df_stats["n_CRIDs"])

# GLM Poisson (n_CRIDs ~ abs_lat)
glm_result = None
if df_stats["n_CRIDs"].sum() > 0 and df_stats["abs_lat"].nunique() > 1:
    X = sm.add_constant(df_stats["abs_lat"])
    y = df_stats["n_CRIDs"].values
    glm_model = sm.GLM(y, X, family=sm.families.Poisson())
    glm_result = glm_model.fit()

print("\n📊 Estatísticas")
print(f"Spearman ρ (n_CRIDs vs |latitude|): {spearman_r:.3f} (p={spearman_p:.3g})")
if glm_result is not None:
    print("GLM Poisson (n_CRIDs ~ |lat|):")
    print(glm_result.summary().tables[1])

# --------------------------------------
# 6) (Opcional) CSV leve de clima/bioma
#    -> Se você tiver um CSV de país com colunas:
#       country, tmean (°C), precip (mm/ano), biome (texto)
#    -> O script integra e faz análises extras sem quebrar se não houver.
# --------------------------------------
print("\n🧩 (Opcional) Envie um CSV de clima/bioma (ou aperte 'Cancelar' para pular).")
try:
    up_aux = files.upload()
    if up_aux:
        aux_path = list(up_aux.keys())[0]
        clima = pd.read_csv(aux_path)
        # nomes esperados (ajuste aqui se usar outros nomes)
        exp_cols = {"country":"country", "tmean":"tmean", "precip":"precip", "biome":"biome"}
        clima_ren = clima.rename(columns={c:exp_cols[c] for c in exp_cols if c in clima.columns})
        assert "country" in clima_ren.columns, "CSV opcional precisa da coluna 'country'."
        world_m = world_m.merge(clima_ren, left_on=NAME_WORLD, right_on="country", how="left")
        # análises simples se tmean/precip existirem
        if {"tmean","precip"}.issubset(world_m.columns):
            sel = world_m["n_CRIDs"] > 0
            if sel.sum() >= 3:
                for var in ["tmean","precip"]:
                    v = world_m.loc[sel, var]
                    corr, pval = stats.spearmanr(v, world_m.loc[sel, "n_CRIDs"])
                    print(f"Spearman ρ (n_CRIDs vs {var}): {corr:.3f} (p={pval:.3g})")
except Exception:
    pass  # segue sem clima

# --------------------------------------
# 7) Plots
# --------------------------------------
# 7a) Mapa (Robinson) de n_CRIDs por país
world_plot = world_m.to_crs(ROBIN_CRS)
fig, ax = plt.subplots(figsize=(20, 10))
ax.set_title("Diversity of analogue series (CRIDs) by country", fontsize=15, pad=12)
ax.axis("off")
# usar viridis; países sem amostra ficam cinza claro
world_plot.plot(ax=ax, column="n_CRIDs", cmap="viridis", linewidth=0.2, edgecolor="white",
                legend=True, legend_kwds={"label":"# of unique CRIDs"},
                missing_kwds={"color": MISSING_COLOR, "edgecolor":"white", "hatch": None})
map_png = "map_diversity_crids_robinson.png"
plt.tight_layout(); plt.savefig(map_png, dpi=DPI_OUT, bbox_inches="tight"); plt.show()

# 7b) Dispersão n_CRIDs vs |latitude| com linha de tendência (GLM esperado)
fig, ax = plt.subplots(figsize=(9,6))
ax.scatter(world_m["abs_lat"], world_m["n_CRIDs"], alpha=0.9)
ax.set_xlabel("|Latitude| (degrees)")
ax.set_ylabel("# unique CRIDs")
ax.set_title("CRID diversity vs distance to Equator")
# curva GLM Poisson se disponível
if glm_result is not None:
    xx = np.linspace(world_m["abs_lat"].min(), world_m["abs_lat"].max(), 200)
    Xp = sm.add_constant(xx)
    mu = glm_result.predict(Xp)
    ax.plot(xx, mu, linewidth=2)
# anotação estatística
ax.text(0.02, 0.98, f"Spearman ρ={spearman_r:.2f}, p={spearman_p:.2g}",
        transform=ax.transAxes, ha="left", va="top")
scat_png = "scatter_crids_vs_abs_lat.png"
plt.tight_layout(); plt.savefig(scat_png, dpi=DPI_OUT, bbox_inches="tight"); plt.show()

# 7c) Boxplot por zona latitudinal
order = ["Tropical","Subtropical","Temperate","Boreal/Polar"]
fig, ax = plt.subplots(figsize=(9,6))
world_m.boxplot(column="n_CRIDs", by="LatZone", ax=ax, grid=False)
ax.set_title("CRID diversity by latitudinal zone")
ax.set_xlabel("Latitudinal zone")
ax.set_ylabel("# unique CRIDs")
plt.suptitle("")
box_png = "boxplot_crids_by_lat_zone.png"
plt.tight_layout(); plt.savefig(box_png, dpi=DPI_OUT, bbox_inches="tight"); plt.show()

# --------------------------------------
# 8) Exportar tabela analítica e baixar figuras
# --------------------------------------
out_csv = "crid_diversity_with_geo.csv"
cols_export = [NAME_WORLD, COUNTRY_COL, "n_CRIDs", "lat", "lon", "abs_lat", "LatZone"]
extra_cols = [c for c in ["tmean","precip","biome"] if c in world_m.columns]
world_m[cols_export + extra_cols].to_csv(out_csv, index=False)

from google.colab import files as gfiles
for f in [out_csv, map_png, scat_png, box_png]:
    try:
        gfiles.download(f)
    except Exception as e:
        print(f"Falha ao iniciar download de {f}: {e}")

print("\n✅ Concluído. Arquivos gerados:")
print(f"- {out_csv}")
print(f"- {map_png}")
print(f"- {scat_png}")
print(f"- {box_png}")


📁 Selecione seu arquivo principal (ex.: 'quarto_file_input_world_map.csv')


Saving quarto_file_input_world_map.csv to quarto_file_input_world_map.csv

🌍 Selecione 'ne_110m_admin_0_countries.zip' (Natural Earth)


Saving ne_110m_admin_0_countries.zip to ne_110m_admin_0_countries (3).zip

📊 Estatísticas
Spearman ρ (n_CRIDs vs |latitude|): -0.041 (p=0.766)
GLM Poisson (n_CRIDs ~ |lat|):
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const          3.0320      0.072     42.119      0.000       2.891       3.173
abs_lat       -0.0071      0.002     -3.454      0.001      -0.011      -0.003

🧩 (Opcional) Envie um CSV de clima/bioma (ou aperte 'Cancelar' para pular).
